<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<div align=center>
<font face="IranNastaliq" size=30>
<p></p>
به نام خدا
</font>
<br>
<font color=#FF7500>
دانشگاه صنعتی شریف - دانشکده مدیریت و اقتصاد شریف
</font>
<br>
<font color=blue>
اقتصاد سنجی ۱
</font>
<br>
ترم پاییز سال تحصیلی ۱۴۰۳-۱۴۰۲
<br>
دکتر کوثر یوسفی
</div>
<hr/>
<font color=red size=6>
<p></p>

<div align=center>بخش ششم: مقدمه‌ای بر کرال کردن دادگان</div>
</font>
<div align=center>
نگین جعفری، محمدمهدی جراحی</div>
<hr>
</font></div>


<style type="text/css" scoped>
p{
border: 1px solid #a2a9b1;background-color: #f8f9fa;display: inline-block;
};
</style>

In [1]:
import pandas as pd
import requests
import tqdm
import os

### Crawl 1385 jamiat tafsili 1 Excels

In [2]:
for i in tqdm.tqdm(range(30)):
    file_name = f'./1385-jamiat-tafsili-1/{str(i).zfill(2)}.xls'
    if os.path.exists(file_name):
        continue
    x = requests.get(
        f'https://www.amar.org.ir/Portals/0/census/1385/results/tables/jamiat/tafsili/1/os{str(i).zfill(2)}.xls')
    with open(file_name, "wb") as f:
        f.write(x.content)

100%|████████████████████████████████████████| 30/30 [00:00<00:00, 14279.29it/s]


In [3]:
def clean_unwanted_characters(cell):
    if not isinstance(cell, str):
        return cell
    cell = cell.replace('­', '').replace('‏', '‌')
    cell = cell.replace('ي', 'ی').replace('ئ', 'ی').replace('ى', 'ی').replace('أ', 'ا').replace('ك', 'ک')
    cell = cell.replace('\u200c', ' ')
    cell = cell.replace('\n', '')
    return cell.strip()

In [4]:
sheets = []
for i in tqdm.tqdm(range(30)):
    file_name = f'./1385-jamiat-tafsili-1/{str(i).zfill(2)}.xls'
    xl = pd.ExcelFile(file_name)

    for sheet_name in xl.sheet_names[1:]:
        df = xl.parse(sheet_name)
        sheet_name = clean_unwanted_characters(sheet_name)

        df = df.iloc[4:, :4].set_index(df.columns[0])
        if df.shape[0] != 123:
            print('Error', i, sheet_name)
        df = df.T
        df.insert(loc=0, column='شهرستان', value=[sheet_name, sheet_name, sheet_name])
        df.insert(loc=1, column='جنسیت', value=['جمع', 'مرد', 'زن'])

        sheets.append((sheet_name, df))

result = pd.concat([s[1] for s in sheets])

100%|███████████████████████████████████████████| 30/30 [00:02<00:00, 12.74it/s]


In [5]:
result.rename(columns=clean_unwanted_characters, inplace=True)

In [6]:
result.to_excel('./1385-jamiat.xlsx', index=False)